In [2]:
# distil Bert
!pip install transformers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

# Step 2: Load and preprocess the data
data = pd.read_csv('reddit_input.csv')
X = data['text']
y = data['class']

# Encode class labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Tokenize the sequences
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

X_train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
X_test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

# Step 4: Convert the encodings to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encodings),
    y_train
)).shuffle(100).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test_encodings),
    y_test
)).batch(16)






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.7 MB/s eta 0:00:00


In [5]:
# Step 5: Load and fine-tune the DistilBERT model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.fit(train_dataset, epochs=5, validation_data=test_dataset)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_39', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/5
94/94 [==============================] - 113s 936ms/step - loss: 0.5766 - accuracy: 0.7447 - val_loss: 0.3458 - val_accuracy: 0.8723
Epoch 2/5
94/94 [==============================] - 88s 938ms/step - loss: 0.2638 - accuracy: 0.8993 - val_loss: 0.2758 - val_accuracy: 0.8989
Epoch 3/5
94/94 [==============================] - 89s 951ms/step - loss: 0.1502 - accuracy: 0.9493 - val_loss: 0.2750 - val_accuracy: 0.9122
Epoch 4/5
94/94 [==============================] - 93s 991ms/step - loss: 0.0960 - accuracy: 0.9700 - val_loss: 0.3205 - val_accuracy: 0.8936
Epoch 5/5
94/94 [==============================] - 93s 991ms/step - loss: 0.0820 - accuracy: 0.9760 - val_loss: 0.4238 - val_accuracy: 0.8830


In [4]:
from sklearn.metrics import classification_report

# Step 6: Evaluate the model on the test dataset
y_pred = model.predict(test_dataset)
y_pred = tf.argmax(y_pred.logits, axis=-1)

# Step 7: Generate the classification report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

24/24 [==============================] - 10s 288ms/step
              precision    recall  f1-score   support

        gpt3       0.88      0.79      0.83       116
       human       0.99      0.97      0.98       137
 instructgpt       0.82      0.91      0.86       123

    accuracy                           0.90       376
   macro avg       0.90      0.89      0.89       376
weighted avg       0.90      0.90      0.90       376

